In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob
from datetime import datetime
from astropy import units as u
from astropy.coordinates import SkyCoord, EarthLocation, builtin_frames, Angle
from astropy.coordinates.erfa_astrom import ErfaAstromInterpolator, erfa_astrom
from astropy.time import Time
from bokeh.io import output_notebook, output_file
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Whisker, Step, Title, LinearColorMapper, BasicTicker, ColorBar
from bokeh.layouts import row
from bokeh.palettes import Inferno256, Greys9
import pyirf
from pyirf.spectral import CRAB_MAGIC_JHEAP2015
from lstchain.reco.utils import  get_effective_time

ModuleNotFoundError: No module named 'lstchain'

In [4]:
output_notebook()

NameError: name 'output_notebook' is not defined

In [3]:
filenames = glob.glob('/nfs/cta-ifae/moralejo/CTA/LST/RealData/DL2/lstchain_v0.7/20201120/dl2_*Run?????.h5')
filenames.sort()

dummy1 = []
dummy2 = []

columns = ['intensity', 'log_intensity', 'length', 'width', 'alt_tel', 'az_tel', 'dragon_time', 'event_type', 
           'delta_t', 'reco_energy', 'reco_src_x', 'reco_src_y', 'reco_alt', 'reco_az', 'gammaness']

# effective time in seconds:
effective_time = 0

for file in filenames:
    print(file)
    tt = pd.read_hdf(file, '/dl2/event/telescope/parameters/LST_LSTCam').astype('float32')[columns]
    # Remove dragon_time and read it again with the original float64 precision:
    tt.drop('dragon_time', axis=1)
    tt['dragon_time'] = pd.read_hdf(file, '/dl2/event/telescope/parameters/LST_LSTCam')['dragon_time'] 
    dummy1.append(tt)
    efft, _ = get_effective_time(tt)
    print("    Eff. time: ", efft)
    effective_time += efft.value
    
    dummy2.append(pd.read_hdf(file, 'source_position').astype('float32'))

tot_data = pd.concat(dummy1, ignore_index=True)
source_position = pd.concat(dummy2, ignore_index=True)
print(tot_data.columns)
print(source_position.columns)

/nfs/cta-ifae/moralejo/CTA/LST/RealData/DL2/lstchain_v0.7/20201120/dl2_LST-1.Run02967.h5
    Eff. time:  1150.907389119496 s
/nfs/cta-ifae/moralejo/CTA/LST/RealData/DL2/lstchain_v0.7/20201120/dl2_LST-1.Run02968.h5
    Eff. time:  1134.3777728726045 s
/nfs/cta-ifae/moralejo/CTA/LST/RealData/DL2/lstchain_v0.7/20201120/dl2_LST-1.Run02969.h5
    Eff. time:  1133.3006634662083 s
/nfs/cta-ifae/moralejo/CTA/LST/RealData/DL2/lstchain_v0.7/20201120/dl2_LST-1.Run02970.h5
    Eff. time:  1129.4495074792453 s
/nfs/cta-ifae/moralejo/CTA/LST/RealData/DL2/lstchain_v0.7/20201120/dl2_LST-1.Run02971.h5
    Eff. time:  1138.1310969248639 s
/nfs/cta-ifae/moralejo/CTA/LST/RealData/DL2/lstchain_v0.7/20201120/dl2_LST-1.Run02972.h5
    Eff. time:  1133.7806966865987 s
/nfs/cta-ifae/moralejo/CTA/LST/RealData/DL2/lstchain_v0.7/20201120/dl2_LST-1.Run02973.h5
    Eff. time:  1131.119014313473 s
/nfs/cta-ifae/moralejo/CTA/LST/RealData/DL2/lstchain_v0.7/20201120/dl2_LST-1.Run02974.h5
    Eff. time:  1138.7933496260

In [5]:
effective_time

12463.508842377094

In [6]:
tot_data = tot_data[(tot_data['event_type']==32)]  #to ignore the calibration events
tot_data.head()

,intensity,log_intensity,length,width,alt_tel,az_tel,dragon_time,event_type,delta_t,reco_energy,reco_src_x,reco_src_y,reco_alt,reco_az,gammaness
0,5652.798828,3.752264,1.178680,1.120216,1.013358,1.63456,1.605918e+09,32.0,0.000000,9.790209,-0.179902,-1.696340,1.004027,1.521554,0.109583
1,4437.973633,3.647185,0.549580,0.340907,1.013358,1.63456,1.605918e+09,32.0,0.000038,6.950095,-1.905059,-0.633798,0.944950,1.596002,0.020000
2,34.195110,1.533964,0.078747,0.028392,1.013358,1.63456,1.605918e+09,32.0,0.000339,0.197329,-1.104301,-0.686172,0.973466,1.591013,0.347000
3,87.852585,1.943755,0.089208,0.041080,1.013358,1.63456,1.605918e+09,32.0,0.000384,0.059509,-0.180538,-1.002562,1.005894,1.567641,0.285667
4,235.864075,2.372662,0.187985,0.095839,1.013359,1.63456,1.605918e+09,32.0,0.000320,0.768756,-1.702260,0.231787,0.952513,1.648816,0.088667


In [7]:
source_position.head()

,src_x,src_y
0,0.107901,0.156028
1,0.107901,0.156028
2,0.107901,0.156028
3,0.107901,0.156028
4,0.107901,0.156028


In [8]:
tot_data.shape

(46076394, 15)

In [9]:
source_position.shape

(47323819, 2)

In [10]:
N, b = np.histogram(np.array(tot_data['delta_t']), bins = 200)  #distribution of the variable delta_t
fig = figure(plot_width=450, plot_height=300)
fig.quad(top=N, bottom=0, left=b[:-1], right=b[1:])
show(fig)

## Calculate the equatorial coordinates (right ascension and declination) of the events + make a skymap of the reconstructed directions of the events in these coordinates.

In [11]:
def utc(df):   #function that calculates the utc time of the events  
    utc_time = []
    for i in range(df.shape[0]):
        idx = list(df.columns).index('dragon_time')
        a = datetime.utcfromtimestamp(df.iloc[i,idx])   #idx corresponds to the index of the column 'dragon_time'
        utc_time.append(a)
        
    return utc_time

In [12]:
def equatorial_coords(df):   #function that calculates the equatorial coordinates (ra and dec) of the events from the horizontal coordinates 
    utc_time = []
    for i in range(df.shape[0]):
        idx = list(df.columns).index('dragon_time')
        a = datetime.utcfromtimestamp(df.iloc[i,idx])   #idx corresponds to the index of the column 'dragon_time'
        utc_time.append(a)

    df = df.assign(utc_time = utc_time)   #add new column to the data frame with the UTC observation times of the events

    loc = EarthLocation(lat = 28.76152611*u.deg, lon = -17.89149701*u.deg, height = 2184*u.m)   #location of the telescope (CTA LST1: Roque de los Muchachos)

    hor_coords = SkyCoord(alt = df['reco_alt'], az = df['reco_az'], frame = 'altaz', unit = 'rad', 
                          obstime = df['utc_time'], location = loc)   #horizontal coordinates of the events
    eq_coords = hor_coords.icrs   #equatorial coordinates of the events
    
    return eq_coords

In [18]:
data = tot_data[(tot_data['gammaness']>0.8)]

eq_coords = equatorial_coords(data)
data = data.assign(utc_time = utc(data))   #add new column to the data frame with the UTC observation times of the events
data = data.assign(RA = eq_coords.ra)   #add columns with right ascension and declination (equatorial coordinates) of the events
data = data.assign(DEC = eq_coords.dec)

# skymap (RA, dec) de las posiciones reconstruidas de los sucesos (ya se ha aplicado el corte de gammaness > 0.8):
counts, bins_x, bins_y = np.histogram2d(data['RA'], data['DEC'], bins=250)
fig = figure(x_range=(min(bins_x), max(bins_x)), y_range=(min(bins_y), max(bins_y)), plot_width=500, plot_height=300)
fig.image(image=[counts], x=bins_x[0], y=bins_y[0], dw=bins_x[-1] - bins_x[0], dh=bins_y[-1] - bins_y[0], palette=Inferno256) 
fig.xaxis.axis_label = 'right ascension (deg)'
fig.yaxis.axis_label = 'declination (deg)'
color_mapper = LinearColorMapper(palette=Inferno256, low=counts.min(), high=counts.max())
color_bar = ColorBar(color_mapper=color_mapper, ticker= BasicTicker(), location=(0,0))
fig.add_layout(color_bar, 'right')
show(fig)

## Ver que hay un exceso significativo de sucesos (gammas) en las coordenadas del Crab y estimar el número de gammas (haciendo "aperture photometry"), tomando 3 zonas off creando una cruz con la posición del Crab (para tener más estadística para calcular el fondo).

In [13]:
def off_zones(df, source_df):   #function that defines the three mentioned off zones and returns the angular separation between the position of each event and each off zone. INPUTS: dataframe with all the events, and a dataframe with the position of the source (Crab)
    src_x = source_df['src_x']/28   #to convert the source_position into radians, we divide by the focal distance (28 m), and then convert to degrees
    src_y = source_df['src_y']/28
    
    reco_src_x = df['reco_src_x']/28  #reconstructed position of the detected events in radians
    reco_src_y = df['reco_src_y']/28
    
    theta2_on = (np.sqrt((src_x - reco_src_x)**2 + (src_y - reco_src_y)**2)*180/np.pi)**2  #angular separation (in degrees) between the events and the position of the Crab 
    theta2_off1 = (np.sqrt((-src_x - reco_src_x)**2 + (-src_y - reco_src_y)**2)*180/np.pi)**2  #separation between the events and symmetric position of the Crab wrt the center of the FOV (-src_x, -src_y)
    theta2_off2 = (np.sqrt((-src_y - reco_src_x)**2 + (src_x - reco_src_y)**2)*180/np.pi)**2   #separation wrt position (-src_y, src_x)
    theta2_off3 = (np.sqrt((src_y - reco_src_x)**2 + (-src_x - reco_src_y)**2)*180/np.pi)**2   #separation wrt position (src_y, -src_x)
   
    return theta2_on, theta2_off1, theta2_off2, theta2_off3 

In [14]:
def gamma_excess(df, source_position):    #function that returns the excess of gamma rays of a given dataframe 
    idx = df.index
    theta2 = off_zones(df, source_position.loc[idx])
    
    n = [0]*4
    b = [0]*4
    colors = ['blue', 'orange', 'green', 'magenta']
    fig1 = figure(plot_width=600, plot_height=400, x_range=(0,2.5))
    for i in range(4):
        n[i], b[i] = np.histogram(np.array(theta2[i]), bins=750)
        
        source = ColumnDataSource(dict(x=(b[i][1:]+b[i][:-1])/2, y=n[i]))
        glyph1 = Step(x='x', y='y', line_color=colors[i], mode='center')
        fig1.add_glyph(source, glyph1)

        base = (b[i][1:]+b[i][:-1])/2
        lower = n[i]-np.sqrt(n[i])/2
        upper = n[i]+np.sqrt(n[i])/2
        source_error = ColumnDataSource(data=dict(base=base, lower=lower, upper=upper))
        w=Whisker(source=source_error, base='base', upper='upper', lower='lower', line_color='gray')
        w.upper_head.line_color = 'gray'
        w.lower_head.line_color = 'gray'
        fig1.add_layout(w)
    
    fig1.xaxis.axis_label = 'theta²'
    fig1.yaxis.axis_label = 'counts'
    show(fig1)

    excess_gammas = n[0][0]+n[0][1]-np.mean([n[1][0],n[2][0],n[3][0]])-np.mean([n[1][1],n[2][1],n[3][1]])
    return print('Excess of gamma rays: {}'. format(excess_gammas))

## Obtain the distribution of theta², width, length and intensity of (only) the gamma rays.

#### *** ERROR PROPAGATION: 

($n_1$, $n_2$, $n_3$, $n_4$): histogram bins (for each position) 

($\sqrt{n_1}$, $\sqrt{n_2}$, $\sqrt{n_3}$, $\sqrt{n_4}$): errors of each "variable" ($n_i$)

new variable (function of $(n_1, n_2, n_3, n_4)$): $n(n_1, n_2, n_3, n_4)=n_1-\frac{1}{3}(n_2+n_3+n_4)$

Covariance matrix of the variables $n_i$ (since $\sigma_i=\sqrt{n_i}$ and $cov(x_i,x_i)=\sigma_i²$): 

\begin{equation*}
V = cov[n_i,n_j]=
\begin{pmatrix}
n_1 &  &  & \\
& n_2 &  & \\
 &  & n_3 &  \\
 &  &  &  n_4
\end{pmatrix}
\end{equation*}

To obtain the variance $U$ of the variable $n$, we use error propagation: 
$$U = \sum_{k,l=1}^n \frac{\partial n}{\partial n_i} \frac{\partial n}{\partial n_j} V_{kl}$$
$$A_1 = \frac{\partial n}{\partial n_1}=1  ; \quad A_2=A_3=A_4=-1/3 \quad \Rightarrow U=V_{11}+\frac{1}{9}(V_{22}+V_{33}+V_{44})$$

In [15]:
def dfs(df):   
    idx = df.index
    theta2 = off_zones(df, source_position.loc[idx])
    
    gammas = df.iloc[np.where(theta2[0]<0.05)]   #dataframe that contains only the events 0.05 degrees around the position of the crab (gammas + bkg of cosmic rays)
    bkg2 = df.iloc[np.where(theta2[1]<0.05)]   #dataframe that contains only the events 0.05 degrees around the off zone 1 (bkg of cosmic rays)
    bkg3 = df.iloc[np.where(theta2[2]<0.05)]   #dataframe that contains only the events 0.05 degrees around the off zone 2 (bkg of cosmic rays)
    bkg4 = df.iloc[np.where(theta2[3]<0.05)]   #dataframe that contains only the events 0.05 degrees around the off zone 3 (bkg of cosmic rays)
    
    dfs = [gammas, bkg2, bkg3, bkg4]
    return dfs

In [16]:
def distribution(df, parameter, bins, xlims = None):   #function that returns the distribution of a given parameter using a given dataframe (with a given cut in gammaness)  
    idx = list(df.columns).index(parameter)
    par = df.columns[idx] 
    
    n = [0]*4
    b = [0]*4
    colors = ['blue', 'orange', 'green', 'magenta']
    fig1 = figure(title = 'Distribution of {} of the gamma rays + background of cosmic rays'.format(par),
                  plot_width=450, plot_height=300, x_range=xlims)
    for i in range(4):
        names = dfs(df)
        name = names[i]
        n[i], b[i] = np.histogram(name[par], bins=bins, range = (xlims))
        
        source = ColumnDataSource(dict(x=(b[i][1:]+b[i][:-1])/2, y=n[i]))
        glyph1 = Step(x='x', y='y', line_color=colors[i], mode='center')
        fig1.add_glyph(source, glyph1)

        base = (b[i][1:]+b[i][:-1])/2
        lower = n[i]-np.sqrt(n[i])/2
        upper = n[i]+np.sqrt(n[i])/2
        source_error = ColumnDataSource(data=dict(base=base, lower=lower, upper=upper))
        w=Whisker(source=source_error, base='base', upper='upper', lower='lower', line_color='gray')
        w.upper_head.line_color = 'gray'
        w.lower_head.line_color = 'gray'
        fig1.add_layout(w)
    
    fig1.xaxis.axis_label = parameter
    fig1.yaxis.axis_label = 'counts'
    fig1.title.text_font_size = '8pt'
    fig1.title.text_font_style = 'normal'
    
    N = n[0]-(1/3)*(n[1]+n[2]+n[3])  #histogram bins of only the gamma rays (without the bkg of cosmic rays)
    err_N = np.sqrt(n[0]+1/9*(n[1]+n[2]+n[3]))  #error of N, obtained by error propagation
    width = b[1][1] - b[1][0]
    
    fig2 = figure(title = 'Distribution of {} of the gamma rays'.format(par),
                  plot_width=450, plot_height=300, x_range=xlims)
    fig2.quad(top=N, bottom=0, left=b[1][:-1], right=b[1][1:])
    base = (b[1][1:]+b[1][:-1])/2
    lower = N - err_N
    upper = N + err_N
    source_error = ColumnDataSource(data=dict(base=base, lower=lower, upper=upper))
    w=Whisker(source=source_error, base='base', upper='upper', lower='lower', line_color='gray')
    w.upper_head.line_color = 'gray'
    w.lower_head.line_color = 'gray'
    fig2.add_layout(w)

    fig2.xaxis.axis_label = parameter
    fig2.yaxis.axis_label = 'counts'
    fig2.title.text_font_size = '8pt'
    fig2.title.text_font_style = 'normal'
    fig2.add_layout(Title(text='(with the background of cosmic rays substracted)', text_font_style='normal', text_font_size='8pt'), 'above')
    
    show(row(fig1, fig2))

In [56]:
# To compare with the MC simulated gamma rays, we represent on the y axis the detected RATE of gamma rays coming from the 
#Crab (number of gamma rays events in each bin/ effective time of observation): 

def dist_rate(df, parameter, bins, t_eff, xlims = None):  #function that returns the distribution of a given parameter representing in the y axis the detected rate of gamma rays coming from the Crab  
    idx = list(df.columns).index(parameter)
    par = df.columns[idx] 
    
    n = [0]*4
    b = [0]*4
    r = [0]*4
    err_r = [0]*4
    colors = ['blue', 'orange', 'green', 'magenta']
    fig1 = figure(title = 'Distribution of {} of the gamma rays + background of cosmic rays'.format(par),
                  plot_width=450, plot_height=300, x_range=xlims)
    
    for i in range(4):
        name = dfs(df)[i]
        n[i], b[i] = np.histogram(name[par], bins=bins, range = (xlims))
        r[i] = n[i]/t_eff  #rate detected of events
        err_r[i] = 1/t_eff*np.sqrt(n[i])
        
        source = ColumnDataSource(dict(x=(b[i][1:]+b[i][:-1])/2, y=r[i]))
        #glyph1 = Step(x='x', y='y', line_color=colors[i], mode='center')
        #fig1.add_glyph(source, glyph1)
        fig1.circle((b[i][1:]+b[i][:-1])/2, r[i], size = 3.5, color = colors[i], alpha = 0.5)
        
        base = (b[i][1:]+b[i][:-1])/2
        lower = r[i]-err_r[i]/2
        upper = r[i]+err_r[i]/2
        source_error = ColumnDataSource(data=dict(base=base, lower=lower, upper=upper))
        w=Whisker(source=source_error, base='base', upper='upper', lower='lower', line_color='gray')
        w.upper_head.line_color = 'gray'
        w.lower_head.line_color = 'gray'
        fig1.add_layout(w)
    
    fig1.xaxis.axis_label = parameter
    fig1.yaxis.axis_label = 'rate of detected events (s⁻¹)'
    fig1.title.text_font_size = '8pt'
    fig1.title.text_font_style = 'normal'
    
    R = r[0]-(1/3)*(r[1]+r[2]+r[3])  #histogram bins of only the gamma rays (without the bkg of cosmic rays)
    err_R = np.sqrt(1/t_eff*(r[0]+1/9*(r[1]+r[2]+r[3])))  #error of N, obtained by error propagation
    width = b[1][1] - b[1][0]
    
    fig2 = figure(title = 'Distribution of {} of the gamma rays'.format(par),
                  plot_width=450, plot_height=300, x_range=xlims)
    fig2.quad(top=R, bottom=0, left=b[1][:-1], right=b[1][1:])
    base = (b[1][1:]+b[1][:-1])/2
    lower = R - err_R
    upper = R + err_R
    source_error = ColumnDataSource(data=dict(base=base, lower=lower, upper=upper))
    w=Whisker(source=source_error, base='base', upper='upper', lower='lower', line_color='gray')
    w.upper_head.line_color = 'gray'
    w.lower_head.line_color = 'gray'
    fig2.add_layout(w)

    fig2.xaxis.axis_label = parameter
    fig2.yaxis.axis_label = 'rate of detected gamma ray events (s⁻¹)'
    fig2.title.text_font_size = '8pt'
    fig2.title.text_font_style = 'normal'
    fig2.add_layout(Title(text='(with the background of cosmic rays substracted)', text_font_style='normal', text_font_size='8pt'), 'above')
    
    show(row(fig1, fig2))

In [33]:
def hist_intensity(df, parameter, bins):  #function that returns the plot of a given parameter in bins of intensity
    idx = list(df.columns).index(parameter)
    par = df.columns[idx] 
    
    counts = []
    for i in range(4):
        name = dfs(df)[i]
        (counts1, bins_x1, bins_y1) = np.histogram2d(name[par], name['intensity'], bins = bins)
        counts.append(counts1)
        
    counts_gamma = counts[0] - (1/3)*(counts[1] + counts[2] + counts[3])
    
    fig = figure(title='Intensity of the gamma ray events as a function of the {}'.format(parameter), plot_width=450, plot_height=400, x_range=(min(bins_x1), max(bins_x1)), y_range=(min(bins_y1), max(bins_y1)))
    fig.image(image=[np.transpose(counts_gamma)], x=bins_x1[0], y=bins_y1[0], dw=bins_x1[-1] - bins_x1[0], 
              dh=bins_y1[-1] - bins_y1[0])
    fig.xaxis.axis_label = parameter
    fig.yaxis.axis_label = 'intensity'
    fig.title.text_font_size = '8pt'
    fig.title.text_font_style = 'normal'
    
    color_mapper = LinearColorMapper(palette=Greys9, low=counts_gamma.min(), high=counts_gamma.max())
    color_bar = ColorBar(color_mapper=color_mapper, ticker= BasicTicker(), location=(0,0))
    fig.add_layout(color_bar, 'right')
    
    show(fig)

# We apply these functions to the data separated in intervals of intensity: <200, 200-500 and  >500, and with a cut in gammaness > 0.2.

In [76]:
tot_data = tot_data.assign(theta2 = off_zones(tot_data, source_position)[0])  #add a column with the values of theta2 to the dataframe

In [91]:
data1 = tot_data[(tot_data['gammaness']>0.2) & (tot_data['intensity']<200)]
data2 = tot_data[(tot_data['gammaness']>0.2) &  (tot_data['intensity']>200) & (tot_data['intensity']<500)]
data3 = tot_data[(tot_data['gammaness']>0.2) &  (tot_data['intensity']>500)]

In [20]:
gamma_excess(data1, source_position)

Excess of gamma rays: -15455.0


In [21]:
gamma_excess(data2, source_position)

Excess of gamma rays: -141.66666666667152


In [22]:
gamma_excess(data3, source_position)

Excess of gamma rays: 3557.666666666667


In [97]:
# distribution of theta2:
def dist_theta2(df, bins, t_eff, xlims = None): 
    fig1 = figure(title = 'Distribution of theta2 of the gamma rays + background of cosmic rays',
                  plot_width=450, plot_height=300, x_range=xlims)

    n, b = np.histogram(df['theta2'], bins=bins, range = (xlims))
    r = n/t_eff  #rate detected of events
    err_r = 1/t_eff*np.sqrt(n)

    source = ColumnDataSource(dict(x=(b[1:]+b[:-1])/2, y=r))
    glyph1 = Step(x='x', y='y', line_color = 'blue', mode='center')
    fig1.add_glyph(source, glyph1)
    #fig1.circle((b[i][1:]+b[i][:-1])/2, r[i], size = 3.5, color = colors[i], alpha = 0.5)

    base = (b[1:]+b[:-1])/2
    lower = r-err_r/2
    upper = r+err_r/2
    source_error = ColumnDataSource(data=dict(base=base, lower=lower, upper=upper))
    w=Whisker(source=source_error, base='base', upper='upper', lower='lower', line_color='gray')
    w.upper_head.line_color = 'gray'
    w.lower_head.line_color = 'gray'
    fig1.add_layout(w)

    fig1.xaxis.axis_label = 'theta2'
    fig1.yaxis.axis_label = 'rate of detected events (s⁻¹)'
    fig1.title.text_font_size = '8pt'
    fig1.title.text_font_style = 'normal'
    
    show(fig1)

In [98]:
dist_theta2(tot_data, 100, effective_time)

In [99]:
dist_theta2(data1, 100, effective_time)

### Intensity < 200

In [57]:
dist_rate(data1, 'width', 100, effective_time)

In [58]:
dist_rate(data1, 'length', 200, effective_time)

In [59]:
dist_rate(data1, 'log_intensity', 60, effective_time)

In [69]:
hist_intensity(data1, 'width', 100)

In [70]:
hist_intensity(data1, 'length', 100)

### Intensity between 200 and 500

In [60]:
dist_rate(data2, 'width', 100, effective_time)

In [63]:
dist_rate(data2, 'length', 200, effective_time)

In [65]:
dist_rate(data2, 'log_intensity', 50, effective_time)

In [71]:
hist_intensity(data2, 'width', 100)

In [72]:
hist_intensity(data2, 'length', 100)

### Intensity  > 500

In [66]:
dist_rate(data3, 'width', 100, effective_time)

In [67]:
dist_rate(data3, 'length', 100, effective_time)

In [75]:
dist_rate(data3, 'log_intensity', 100, effective_time)

In [73]:
hist_intensity(data3, 'width', 100)

In [74]:
hist_intensity(data3, 'length', 100)